In [ ]:
%pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import numpy as np
import random
from sklearn.datasets import make_regression
from datetime import datetime

x, y = make_regression(n_samples = 100,n_features = 5,n_targets = 1, random_state = 1)
dfx = pd.DataFrame(x, columns=['A','B','C','D','E'])
dfy = pd.DataFrame(y, columns=['label'])

# Create an integer column, a datetime column, a categorical column and a string column to demonstrate how they are processed.
dfx['B'] = (dfx['B']).astype(int)
dfx['C'] = datetime(2000,1,1) + pd.to_timedelta(dfx['C'].astype(int), unit='D')
dfx['D'] = pd.cut(dfx['D'] * 10, [-np.inf,-5,0,5,np.inf],labels=['v','w','x','y'])
dfx['E'] = pd.Series(list(' '.join(random.choice(["abc", "d", "ef", "ghi", "jkl"]) for i in range(4)) for j in range(100)))
dataset=TabularDataset(dfx)
print(dfx)

           A  B          C  D                E
0  -0.545774  0 2000-01-01  y    jkl ghi d ghi
1  -0.468674  0 2000-01-02  x  ghi ghi ghi jkl
2   1.767960  0 1999-12-31  v      jkl d ef ef
3  -0.118771  1 2000-01-01  y      d ef jkl ef
4   0.630196  0 1999-12-31  w   jkl ef abc abc
..       ... ..        ... ..              ...
95 -1.182318 -1 2000-01-01  v      ef d ef abc
96  0.562761  0 2000-01-01  v       d jkl ef d
97 -0.797270  0 2000-01-01  w     jkl ef jkl d
98  0.502741  0 1999-12-31  y  abc ghi abc abc
99  2.056356  0 1999-12-30  w    d abc jkl jkl

[100 rows x 5 columns]


In [ ]:
from autogluon.features.generators import AutoMLPipelineFeatureGenerator
auto_ml_pipeline_feature_generator = AutoMLPipelineFeatureGenerator()
auto_ml_pipeline_feature_generator.fit_transform(X=dfx)

,A,B,D,E,C,C.year,C.month,C.day,C.dayofweek,E.char_count,E.symbol_ratio.,__nlp__.abc,__nlp__.ef,__nlp__.ghi,__nlp__.jkl,__nlp__._total_
0,-0.545774,0,3,NaN,946684800000000000,2000,1,1,5,4,2,0,0,2,1,2
1,-0.468674,0,2,4,946771200000000000,2000,1,2,6,6,0,0,0,3,1,2
2,1.767960,0,0,NaN,946598400000000000,1999,12,31,4,2,4,0,2,0,1,2
3,-0.118771,1,3,NaN,946684800000000000,2000,1,1,5,2,4,0,2,0,1,2
4,0.630196,0,1,NaN,946598400000000000,1999,12,31,4,5,1,2,1,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-1.182318,-1,0,3,946684800000000000,2000,1,1,5,2,4,1,2,0,0,2
96,0.562761,0,0,NaN,946684800000000000,2000,1,1,5,1,5,0,1,0,1,2
97,-0.797270,0,1,NaN,946684800000000000,2000,1,1,5,3,3,0,1,0,2,2
98,0.502741,0,3,NaN,946598400000000000,1999,12,31,4,6,0,3,0,1,0,2


In [ ]:
df = pd.concat([dfx, dfy], axis=1)
predictor = TabularPredictor(label='label')
predictor.fit(df, hyperparameters={'GBM' : {}}, feature_generator=auto_ml_pipeline_feature_generator)

No path specified. Models will be saved in: "AutogluonModels/ag-20251026_084005"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Memory Avail:       11.40 GB / 12.67 GB (90.0%)
Disk Space Avail:   179.74 GB / 225.83 GB (79.6%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory and 32+ GB GPU memory.
	presets=

In [ ]:
print(len(set(dfx['B'])))

5


In [ ]:
dfx["B"] = dfx["B"].astype("category")
auto_ml_pipeline_feature_generator = AutoMLPipelineFeatureGenerator()
auto_ml_pipeline_feature_generator.fit_transform(X=dfx)

Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11233.53 MB
	Train Data (Original)  Memory Usage: 0.01 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
		Fitting DatetimeFeatureGenerator...
		Fitting TextSpecialFeatureGenerator...
			Fitting BinnedFeatureGenerator...
			Fitting DropDuplicatesFeatureGenerator...
		Fitting TextNgramFeatureGenerator...
			Fitting CountVectorizer for text features: ['E']
			CountVectorizer fit with vocabulary size = 4
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types o

,A,B,D,E,C,C.year,C.month,C.day,C.dayofweek,E.char_count,E.symbol_ratio.,__nlp__.abc,__nlp__.ef,__nlp__.ghi,__nlp__.jkl,__nlp__._total_
0,-0.545774,1,3,NaN,946684800000000000,2000,1,1,5,4,2,0,0,2,1,2
1,-0.468674,1,2,4,946771200000000000,2000,1,2,6,6,0,0,0,3,1,2
2,1.767960,1,0,NaN,946598400000000000,1999,12,31,4,2,4,0,2,0,1,2
3,-0.118771,2,3,NaN,946684800000000000,2000,1,1,5,2,4,0,2,0,1,2
4,0.630196,1,1,NaN,946598400000000000,1999,12,31,4,5,1,2,1,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-1.182318,0,0,3,946684800000000000,2000,1,1,5,2,4,1,2,0,0,2
96,0.562761,1,0,NaN,946684800000000000,2000,1,1,5,1,5,0,1,0,1,2
97,-0.797270,1,1,NaN,946684800000000000,2000,1,1,5,3,3,0,1,0,2,2
98,0.502741,1,3,NaN,946598400000000000,1999,12,31,4,6,0,3,0,1,0,2


In [ ]:
dfx.iloc[0] = np.nan
dfx.head()

,A,B,C,D,E
0,NaN,NaN,NaT,NaN,NaN
1,-0.468674,0,2000-01-02,x,ghi ghi ghi jkl
2,1.767960,0,1999-12-31,v,jkl d ef ef
3,-0.118771,1,2000-01-01,y,d ef jkl ef
4,0.630196,0,1999-12-31,w,jkl ef abc abc


In [ ]:
auto_ml_pipeline_feature_generator = AutoMLPipelineFeatureGenerator()
auto_ml_pipeline_feature_generator.fit_transform(X=dfx)

Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11148.23 MB
	Train Data (Original)  Memory Usage: 0.01 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
		Fitting DatetimeFeatureGenerator...
		Fitting TextSpecialFeatureGenerator...
			Fitting BinnedFeatureGenerator...
			Fitting DropDuplicatesFeatureGenerator...
		Fitting TextNgramFeatureGenerator...
			Fitting CountVectorizer for text features: ['E']
			CountVectorizer fit with vocabulary size = 4
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types o

,A,B,D,E,C,C.year,C.month,C.day,C.dayofweek,E.char_count,E.word_count,E.symbol_ratio.,__nlp__.abc,__nlp__.ef,__nlp__.ghi,__nlp__.jkl,__nlp__._total_
0,NaN,NaN,NaN,NaN,946687418181818240,2000,1,1,5,0,0,0,0,0,0,0,0
1,-0.468674,1,2,4,946771200000000000,2000,1,2,6,7,1,1,0,0,3,1,2
2,1.767960,1,0,NaN,946598400000000000,1999,12,31,4,3,1,5,0,2,0,1,2
3,-0.118771,2,3,NaN,946684800000000000,2000,1,1,5,3,1,5,0,2,0,1,2
4,0.630196,1,1,NaN,946598400000000000,1999,12,31,4,6,1,2,2,1,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-1.182318,0,0,3,946684800000000000,2000,1,1,5,3,1,5,1,2,0,0,2
96,0.562761,1,0,NaN,946684800000000000,2000,1,1,5,2,1,6,0,1,0,1,2
97,-0.797270,1,1,NaN,946684800000000000,2000,1,1,5,4,1,4,0,1,0,2,2
98,0.502741,1,3,NaN,946598400000000000,1999,12,31,4,7,1,1,3,0,1,0,2


In [ ]:
from autogluon.features.generators import PipelineFeatureGenerator, CategoryFeatureGenerator, IdentityFeatureGenerator
from autogluon.common.features.types import R_INT, R_FLOAT
mypipeline = PipelineFeatureGenerator(
    generators = [[
        CategoryFeatureGenerator(maximum_num_cat=10),  # Overridden from default.
        IdentityFeatureGenerator(infer_features_in_args=dict(valid_raw_types=[R_INT, R_FLOAT])),
    ]]
)

In [ ]:
mypipeline.fit_transform(X=dfx)

Fitting PipelineFeatureGenerator...
	Available Memory:                    11174.54 MB
	Train Data (Original)  Memory Usage: 0.01 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Unused Original Features (Count: 1): ['C']
		These features were not used to generate any of the output features. Add a feature generator compatible with these features to utilize them.
		Features can also be unused if they carry very little information, such as being categorical but having almost entirely uniqu

,B,D,E,A
0,NaN,NaN,NaN,NaN
1,1,2,4,-0.468674
2,1,0,NaN,1.767960
3,2,3,NaN,-0.118771
4,1,1,NaN,0.630196
...,...,...,...,...
95,0,0,3,-1.182318
96,1,0,NaN,0.562761
97,1,1,NaN,-0.797270
98,1,3,NaN,0.502741
